### Combined Mdoel

In this part, we are going to combine multiple models.

In [1]:
#!pip install librosa
import librosa
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

#!pip install tensorflow[and-cuda]
#!pip install --upgrade keras
os.environ["KERAS_BACKEND"] = "tensorflow"
import keras
import tensorflow as tf

#### Data
Lets preprocess all the data we used in each models(LSTM, RNN and CNN)

In [2]:
data = pd.read_csv("../training_set.csv")
data = data[['affect','phone_position','file_path']]
data.head

<bound method NDFrame.head of        affect                                     phone_position  \
0     sadness  Place phone on the opposite side of the room i...   
1     neutral  Hold your phone next to your face but with the...   
2     neutral  Place phone on the opposite side of the room f...   
3     neutral  Place phone on the opposite side of the room f...   
4     disgust  Place phone 1-2 meters away face up on any sur...   
...       ...                                                ...   
2948  sadness  Place phone as far away as possible while on o...   
2949  sadness  Place phone as far away as possible while on o...   
2950  sadness  Place phone as far away as possible while on o...   
2951     fear  Place phone 1-2 meters away face down on a har...   
2952     fear  Place phone 1-2 meters away face down on a har...   

                                              file_path  
0     /dataset/shout_data_3afd7208-7987-4c73-8e01-58...  
1     /dataset/shout_data_ae6feb35-cf

In [3]:
emotion_dict = {"neutral": 0, "anger": 1, "joy":2, "fear":3, "disgust":4, "surprise": 5, "sadness": 6}
position_dict = {"Hold your phone next to your face with the mic facing your mouth as you would in a phone conversation 1/19": 0,
                "Hold your phone next to your face but with the mic/phone facing away from your face 2/19": 1,
                "Hold your phone next to your face with your hand covering the mic 3/19": 2,
                "Hold your phone next to your hip with the back of phone on your palm 4/19": 3,
                "Hold your phone next to your hip with your hand covering the mic 5/19": 4,
                "Place your phone in your pocket 6/19": 5,
                "Place your phone in a bag and hold it next to your hip 7/19": 6,
                "Place phone 1-2 meters away face up on any surface 8/19": 7,
                "Place phone 1-2 meters away face down on a hard surface 9/19": 8,
                "Place phone 1-2 meters away face down on a soft surface 10/19": 9,
                "Place phone 1-2 meters away in a bag 11/19": 10,
                "Place phone on the opposite side of the room face up on any surface 12/19": 11,
                "Place phone on the opposite side of the room face down on a hard surface 13/19": 12,
                "Place phone on the opposite side of the room face down on a soft surface 14/19": 13,
                "Place phone on the opposite side of the room in a bag 15/19": 14,
                "Place phone as far away as possible while on on the opposite side of a wall from you face up on any surface 16/19": 15,
                "Place phone as far away as possible while on on the opposite side of a wall from you face down on a hard surface 17/19": 16,
                "Place phone as far away as possible while on on the opposite side of a wall from you face down on a soft surface 18/19": 17,
                "Place phone as far away as possible while on on the opposite side of a wall from you in a bag 19/19": 18}

In [4]:
data['affect'] = data['affect'].map(emotion_dict)
data['phone_position'] = data['phone_position'].map(position_dict)
data = data.drop(data[data.phone_position.isna()].index)
data['phone_position'] = data['phone_position'].astype('uint8')
data.head

<bound method NDFrame.head of       affect  phone_position  \
0          6              14   
1          0               1   
2          0              12   
3          0              12   
4          4               7   
...      ...             ...   
2948       6              16   
2949       6              16   
2950       6              16   
2951       3               8   
2952       3               8   

                                              file_path  
0     /dataset/shout_data_3afd7208-7987-4c73-8e01-58...  
1     /dataset/shout_data_ae6feb35-cf7a-4805-8d05-03...  
2     /dataset/shout_data_fdb758a5-9c86-40f7-8870-ca...  
3     /dataset/shout_data_fdb758a5-9c86-40f7-8870-ca...  
4     /dataset/shout_data_739dbf1c-ec21-42db-9e49-3a...  
...                                                 ...  
2948  /dataset/shout_data_f8d1719f-8a4b-4d7a-a2a7-ef...  
2949  /dataset/shout_data_f8d1719f-8a4b-4d7a-a2a7-ef...  
2950  /dataset/shout_data_f8d1719f-8a4b-4d7a-a2a7-ef...  
2951 

In [4]:
FRAME_SIZE = 512 # the size of the frame
HOP_LENGTH = 256 # the distance each frame jumps by
FRAME_LENGTH = 512 # length of a frame

def getFeatures(audioSignal, sample_rate, maxAudioLength):
    audioFile = librosa.util.pad_center(audioSignal, size=maxAudioLength)
    
    # duration of 1 sample
    oneSampleDuration = 1/sample_rate
    
    # duration of audio signal in seconds
    sampleDuration = oneSampleDuration*len(audioFile)
    
    ae, time = getAmplitudeEnvelope(audioFile)
    rms = getRMS(audioFile)
    zcr = getZCR(audioFile)
    
    return time, ae, rms, zcr
    
def getAmplitudeEnvelope(audioSignal):
    amplitudeEnvelope = []
    
    for i in range(0, len(audioSignal), HOP_LENGTH):
        currentFrameAE = max(audioSignal[i:i+FRAME_SIZE])
        amplitudeEnvelope.append(currentFrameAE)
        
    amplitudeEnvelope = np.array(amplitudeEnvelope)
    amplitudeFrames = range(0, amplitudeEnvelope.size)
    time = librosa.frames_to_time(amplitudeFrames, hop_length=HOP_LENGTH)
    
    return amplitudeEnvelope, time
    
def getRMS(audioSignal):
    rms = []
    for i in range(0, len(audioSignal), HOP_LENGTH):
        rms_current = np.sqrt(np.sum(audioSignal[i:i+FRAME_LENGTH]**2) / FRAME_LENGTH)
        rms.append(rms_current)
    return rms

def getZCR(audioSignal):
    return librosa.feature.zero_crossing_rate(audioSignal, frame_length=FRAME_LENGTH, hop_length=HOP_LENGTH)[0]

def calc_sfb(spec, freq, sr):
    freq_range = sr/2
    freq_delta = freq_range / spec.shape[0]
    return int(np.floor(freq / freq_delta))
    
def calc_ber(spec, freq, sr):
    sfb = calc_sfb(spec, freq, sr)
    pSpec = np.abs(spec) ** 2
    pSpec = pSpec.T
    ber = []

    for freqs in pSpec:
        sum_low = np.sum(freqs[:sfb])
        sum_high = np.sum(freqs[sfb:])
        ber_current = sum_low/sum_high
        ber.append(ber_current)

    return np.array(ber)
    
def getFrequencyFeatures(audioSignal, sample_rate):
    audio_spec = librosa.stft(audioFile, n_fft=FRAME_SIZE, hop_length=HOP_LENGTH)
    centroid = librosa.feature.spectral_centroid(y=audioFile, sr=sample_rate, n_fft=FRAME_SIZE, hop_length=HOP_LENGTH)[0]
    bandwidth = librosa.feature.spectral_bandwidth(y=audioFile, sr=sample_rate, n_fft=FRAME_SIZE, hop_length=HOP_LENGTH)[0]
    ber = calc_ber(audio_spec, 2000, sample_rate)
    return centroid, bandwidth, ber

In [5]:
def getPitch(audioSignal, sample_rate):
    pitches, magnitudes = librosa.piptrack(y=audioSignal, sr=sample_rate)
    max_magnitudes_index = np.argmax(magnitudes, axis=0)
    pitch_freqs = librosa.fft_frequencies(sr=sample_rate)
    pitch_values = pitch_freqs[max_magnitudes_index]
    # Calculate the minimum and maximum pitch values across all frames
    min_pitch = np.min(pitch_values)
    max_pitch = np.max(pitch_values)

    # Calculate the pitch range and average
    pitch_range = max_pitch - min_pitch
    pitch_average = np.mean(pitch_values)
    
    return pitch_values, pitch_range, pitch_average

In [49]:
rgb = []
zeroCrossingRateList = []
amplitudeEnvelopeList = []
rootMeanSquareEnergyList = []
timings = []
spectralCentroidList = []
spectralBandwidthList = []
bandEnergyRatioList = []
audioFileLengths = []
pitch = []
pitchRange = []
pitchAverage = []

audioFileLengths = []
for fileDir in data['file_path']:
    audioFile, sample_rate = librosa.load('..'+fileDir)
    audioFileLengths.append(len(audioFile))

maxAudioLength = max(audioFileLengths)

for index, row in data.iterrows():
    position = row['phone_position']
    y, sr = librosa.load('..'+row['file_path'])
    S = librosa.feature.melspectrogram(y=y, sr=sr)
    S_dB = librosa.power_to_db(S, ref=np.max)
    fig, ax = plt.subplots()
    fig.set_size_inches(3,2)
    image = librosa.display.specshow(S_dB, sr=sr,ax=ax)
    fig.canvas.draw()
    
    # Convert the canvas to a raw RGB buffer
    buf = fig.canvas.tostring_rgb()
    ncols, nrows = fig.canvas.get_width_height()
    image = np.frombuffer(buf, dtype=np.uint8).reshape(nrows, ncols, 3)

    rgb.append(image)
    plt.close()
    
    # Get Features for LSTM
    time, ae, rmse, zcr = getFeatures(y, sr, maxAudioLength)
    zeroCrossingRateList.append(zcr)
    amplitudeEnvelopeList.append(ae)
    rootMeanSquareEnergyList.append(rmse)
    timings.append(time)

    #Get Features for RNN
    centroid, bandwidth, ber = getFrequencyFeatures(y, sr)
    spectralCentroidList.append(centroid)
    spectralBandwidthList.append(bandwidth)
    bandEnergyRatioList.append(ber)

    #Get features about pitch
    pitch_values, pitch_range, pitch_average = getPitch(y,sr)
    pitch.append(np.array(pitch_values))
    pitchRange.append(pitch_range)
    pitchAverage.append(pitch_average)

max_pitch = max(map(len, data['Pitch']))
for i in range(len(pitch)):
    pitch[i] = librosa.util.fix_length(pitch[i], size=max_pitch)

data['rgb'] = rgb
data["Timings"] = timings
data["Amplitude_Envelope"] = amplitudeEnvelopeList
data["Root_Mean_Square_Energy"] = rootMeanSquareEnergyList
data["Zero-Crossing_Rate"] = zeroCrossingRateList
data["Spectral Centroid"] = spectralCentroidList
data["Spectral Bandwidth"] = spectralBandwidthList
data["Band Energy Ratio"] = bandEnergyRatioList
data["Pitch"] = pitch
data["Pitch_Range"] = pitchRange
data["Pitch_Average"] = pitchAverage

data.head()

[1614.99023438 1614.99023438 1518.09082031 1528.85742188 1518.09082031
 1518.09082031 1518.09082031 1614.99023438 1970.28808594 1981.0546875
  979.76074219 2271.75292969  549.09667969  559.86328125  559.86328125
  516.796875    506.03027344 1518.09082031 1518.09082031 1518.09082031
 1518.09082031 1518.09082031 1012.06054688  613.69628906  613.69628906
  613.69628906  645.99609375  645.99609375  645.99609375  645.99609375
  742.89550781  742.89550781  441.43066406  452.19726562  645.99609375
  645.99609375  473.73046875  452.19726562  441.43066406  785.96191406
  796.72851562  753.66210938 1108.95996094 1119.7265625  1130.49316406
 1141.25976562  775.1953125  1184.32617188 1195.09277344 1205.859375
  861.328125    872.09472656 1518.09082031 1518.09082031 1518.09082031
 1528.85742188 1528.85742188 1571.92382812 1571.92382812  495.26367188
  506.03027344  527.56347656  538.33007812  527.56347656  527.56347656
  516.796875   2885.44921875  441.43066406  785.96191406  785.96191406
  796.728

,affect,phone_position,file_path,rgb,Timings,Amplitude_Envelope,Root_Mean_Square_Energy,Zero-Crossing_Rate,Spectral Centroid,Spectral Bandwidth,Band Energy Ratio,Pitch,Pitch_Range,Pitch_Average
0,6,14,/dataset/shout_data_3afd7208-7987-4c73-8e01-58...,"[[[255, 255, 255], [255, 255, 255], [255, 255,...","[0.0, 0.011609977324263039, 0.0232199546485260...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[7264.776541065969, 5895.177040938616, 3288.64...","[2806.178363314588, 3236.2573043919083, 3395.4...","[0.029246358, 0.18647823, 9.129268, 232.08543,...","[1614.990234375, 1614.990234375, 1518.09082031...",3779.077148,916.046992
1,0,1,/dataset/shout_data_ae6feb35-cf7a-4805-8d05-03...,"[[[255, 255, 255], [255, 255, 255], [255, 255,...","[0.0, 0.011609977324263039, 0.0232199546485260...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[7264.776541065969, 5895.177040938616, 3288.64...","[2806.178363314588, 3236.2573043919083, 3395.4...","[0.029246358, 0.18647823, 9.129268, 232.08543,...","[2293.2861328125, 2293.2861328125, 2282.519531...",3972.875977,1235.374714
2,0,12,/dataset/shout_data_fdb758a5-9c86-40f7-8870-ca...,"[[[255, 255, 255], [255, 255, 255], [255, 255,...","[0.0, 0.011609977324263039, 0.0232199546485260...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[7264.776541065969, 5895.177040938616, 3288.64...","[2806.178363314588, 3236.2573043919083, 3395.4...","[0.029246358, 0.18647823, 9.129268, 232.08543,...","[344.53125, 333.7646484375, 236.865234375, 236...",2648.583984,522.299805
3,0,12,/dataset/shout_data_fdb758a5-9c86-40f7-8870-ca...,"[[[255, 255, 255], [255, 255, 255], [255, 255,...","[0.0, 0.011609977324263039, 0.0232199546485260...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[7264.776541065969, 5895.177040938616, 3288.64...","[2806.178363314588, 3236.2573043919083, 3395.4...","[0.029246358, 0.18647823, 9.129268, 232.08543,...","[247.6318359375, 247.6318359375, 1130.49316406...",3079.248047,612.260742
4,4,7,/dataset/shout_data_739dbf1c-ec21-42db-9e49-3a...,"[[[255, 255, 255], [255, 255, 255], [255, 255,...","[0.0, 0.011609977324263039, 0.0232199546485260...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[7264.776541065969, 5895.177040938616, 3288.64...","[2806.178363314588, 3236.2573043919083, 3395.4...","[0.029246358, 0.18647823, 9.129268, 232.08543,...","[2454.78515625, 2454.78515625, 150.732421875, ...",3800.610352,708.383388


In [50]:
valid = data.sample(n=100)
train = data.drop(valid.index)
print(valid.head)
print(train.head)

<bound method NDFrame.head of       affect  phone_position  \
469        0              12   
2113       5              17   
1124       3              14   
1356       5               6   
1377       1              14   
...      ...             ...   
2057       4               3   
1006       1              14   
1616       3              12   
2791       5               1   
2255       1               4   

                                              file_path  \
469   /dataset/shout_data_7759ff7a-030f-408d-8658-74...   
2113  /dataset/shout_data_88ff6c39-605c-47a6-8042-51...   
1124  /dataset/shout_data_6cd09683-0fb1-47c8-890f-20...   
1356  /dataset/shout_data_b6a3f240-30fc-4fe6-b41d-dd...   
1377  /dataset/shout_data_d4704d21-1e7d-4b72-afa9-db...   
...                                                 ...   
2057  /dataset/shout_data_6aea45e2-f290-40fe-8be6-2d...   
1006  /dataset/shout_data_df512023-095f-4b75-b5cd-3e...   
1616  /dataset/shout_data_9301fc6a-fa94-44d6-aa37-ad..

In [51]:
y_train = train['affect']
y_valid = valid['affect']
y_train = keras.utils.to_categorical(y_train, num_classes=7)
y_valid = keras.utils.to_categorical(y_valid, num_classes=7)
print(y_train.shape)
print(y_valid.shape)

(2847, 7)
(100, 7)


### CNN with Melspectrogram (RGB)

In [88]:
#data for CNN
x_train_cnn = np.asarray(train['rgb'])
x_train_cnn = np.stack(x_train_cnn, axis=0)
x_valid_cnn = np.asarray(valid['rgb'])
x_valid_cnn = np.stack(x_valid_cnn, axis = 0)
print(x_train_cnn.shape)
print(x_valid_cnn.shape)

(2847, 200, 300, 3)
(100, 200, 300, 3)


In [90]:
cnn = keras.Sequential()

cnn.add(keras.layers.Input(shape=(200,300,3)))
cnn.add(keras.layers.Conv2D(64, kernel_size=(5, 5), activation='relu'))
cnn.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))
cnn.add(keras.layers.BatchNormalization())
cnn.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
cnn.add(keras.layers.Dropout(0.10))

cnn.add(keras.layers.Flatten())
cnn.add(keras.layers.Dense(7, activation='softmax'))

cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
cnn.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 196, 296, 64)      4864      
                                                                 
 conv2d_3 (Conv2D)           (None, 194, 294, 32)      18464     
                                                                 
 batch_normalization_1 (Batc  (None, 194, 294, 32)     128       
 hNormalization)                                                 
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 97, 147, 32)      0         
 2D)                                                             
                                                                 
 dropout_13 (Dropout)        (None, 97, 147, 32)       0         
                                                                 
 flatten_9 (Flatten)         (None, 456288)           

In [91]:
y_train_tensor = tf.convert_to_tensor(y_train)
y_valid_tensor = tf.convert_to_tensor(y_valid)

x_train_tensor = tf.convert_to_tensor(x_train_cnn)
x_valid_tensor = tf.convert_to_tensor(x_valid_cnn)

# training the model
cnn.fit(x = x_train_tensor, y = y_train_tensor, batch_size=16, epochs=20, validation_data=(x_valid_tensor, y_valid_tensor))

Epoch 1/20
178/178 [==============================] - 10s 42ms/step - loss: 43.7194 - accuracy: 0.1531 - val_loss: 126.8163 - val_accuracy: 0.2200
Epoch 2/20
178/178 [==============================] - 7s 38ms/step - loss: 22.0852 - accuracy: 0.2603 - val_loss: 29.4800 - val_accuracy: 0.1200
Epoch 3/20
178/178 [==============================] - 7s 39ms/step - loss: 13.5823 - accuracy: 0.4334 - val_loss: 19.2657 - val_accuracy: 0.2100
Epoch 4/20
178/178 [==============================] - 7s 38ms/step - loss: 4.6422 - accuracy: 0.7545 - val_loss: 37.6051 - val_accuracy: 0.1700
Epoch 5/20
178/178 [==============================] - 7s 38ms/step - loss: 2.6517 - accuracy: 0.8563 - val_loss: 53.2927 - val_accuracy: 0.2400
Epoch 6/20
178/178 [==============================] - 7s 39ms/step - loss: 2.5985 - accuracy: 0.8676 - val_loss: 38.3036 - val_accuracy: 0.2100
Epoch 7/20
178/178 [==============================] - 7s 38ms/step - loss: 1.8593 - accuracy: 0.9164 - val_loss: 45.4053 - val_accu

In [98]:
def grayscale(row):
    return(row.dot([[0.299],[0.587],[0.114]]).reshape(200,300).astype('uint8'))

x_train_cnn_gray = []
for row in train['rgb']:
    x_train_cnn_gray.append(grayscale(row))

x_train_cnn_gray = np.asarray(x_train_cnn_gray)
x_train_cnn_gray = np.stack(x_train_cnn_gray, axis=0)

x_valid_cnn_gray = []
for row in valid['rgb']:
    x_valid_cnn_gray.append(grayscale(row))

x_valid_cnn_gray = np.asarray(x_valid_cnn_gray)
x_valid_cnn_gray = np.stack(x_valid_cnn_gray, axis=0)



print(x_train_cnn_gray.shape)
print(x_valid_cnn_gray.shape)

(2847, 200, 300, 1)
(100, 200, 300, 1)


In [101]:
cnn = keras.Sequential()

cnn.add(keras.layers.Input(shape=(200,300,1)))
cnn.add(keras.layers.Conv2D(64, kernel_size=(5, 5), activation='relu'))
cnn.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))
cnn.add(keras.layers.BatchNormalization())
cnn.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
cnn.add(keras.layers.Dropout(0.10))

cnn.add(keras.layers.Flatten())
cnn.add(keras.layers.Dense(7, activation='softmax'))

cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
cnn.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 196, 296, 64)      1664      
                                                                 
 conv2d_7 (Conv2D)           (None, 194, 294, 32)      18464     
                                                                 
 batch_normalization_3 (Batc  (None, 194, 294, 32)     128       
 hNormalization)                                                 
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 97, 147, 32)      0         
 2D)                                                             
                                                                 
 dropout_15 (Dropout)        (None, 97, 147, 32)       0         
                                                                 
 flatten_11 (Flatten)        (None, 456288)           

In [102]:
y_train_tensor = tf.convert_to_tensor(y_train)
y_valid_tensor = tf.convert_to_tensor(y_valid)

x_train_tensor = tf.convert_to_tensor(x_train_cnn_gray)
x_valid_tensor = tf.convert_to_tensor(x_valid_cnn_gray)

# training the model
cnn.fit(x = x_train_tensor, y = y_train_tensor, batch_size=16, epochs=20, validation_data=(x_valid_tensor, y_valid_tensor))

Epoch 1/20
178/178 [==============================] - 7s 34ms/step - loss: 29.0899 - accuracy: 0.1517 - val_loss: 19.4792 - val_accuracy: 0.1800
Epoch 2/20
178/178 [==============================] - 6s 33ms/step - loss: 11.5545 - accuracy: 0.3565 - val_loss: 22.2577 - val_accuracy: 0.1600
Epoch 3/20
178/178 [==============================] - 6s 33ms/step - loss: 3.5404 - accuracy: 0.7102 - val_loss: 21.9162 - val_accuracy: 0.1300
Epoch 4/20
178/178 [==============================] - 6s 33ms/step - loss: 1.8280 - accuracy: 0.8570 - val_loss: 21.9957 - val_accuracy: 0.2500
Epoch 5/20
178/178 [==============================] - 6s 33ms/step - loss: 1.1893 - accuracy: 0.9059 - val_loss: 60.5694 - val_accuracy: 0.1200
Epoch 6/20
178/178 [==============================] - 6s 33ms/step - loss: 0.9698 - accuracy: 0.9354 - val_loss: 30.4993 - val_accuracy: 0.1500
Epoch 7/20
178/178 [==============================] - 6s 33ms/step - loss: 0.9514 - accuracy: 0.9382 - val_loss: 36.6537 - val_accurac

### LSTM with AE, RMS, ZCR

In [85]:
#data for LSTM
x_train_lstm = []
for dataCollection in np.array(train[["Amplitude_Envelope", "Root_Mean_Square_Energy", "Zero-Crossing_Rate"]]):
    AE_data, RMS_data, ZCR_data = dataCollection
    x_train_lstm.append([AE_data, RMS_data, ZCR_data])
x_train_lstm = np.array(x_train_lstm)
print(x_train_lstm.shape)

x_valid_lstm = []
for dataCollection in np.array(valid[["Amplitude_Envelope", "Root_Mean_Square_Energy", "Zero-Crossing_Rate"]]):
    AE_data, RMS_data, ZCR_data = dataCollection
    x_valid_lstm.append([AE_data, RMS_data, ZCR_data])
x_valid_lstm = np.array(x_valid_lstm)
print(x_valid_lstm.shape)


(2847, 3, 639)
(100, 3, 639)


In [86]:
### https://faroit.com/keras-docs/1.0.1/getting-started/sequential-model-guide/#the-merge-layer
data_dimension = 3
timesteps = len(data["Timings"][0])
numberClasses = 7

# data from audio clips LSTM
lstmModel = keras.Sequential()
lstmInput = keras.layers.Input(shape=(data_dimension, timesteps))
lstmModel.add(lstmInput)
lstmModel.add(keras.layers.LSTM(128, return_sequences=True))
lstmModel.add(keras.layers.LSTM(64, return_sequences=True))
lstmModel.add(keras.layers.LSTM(32))
lstmModel.add(keras.layers.Dropout(0.1))

# adding final dense layers
lstmModel.add(keras.layers.Dense(32, activation="relu"))
lstmModel.add(keras.layers.Dense(16, activation="relu"))
lstmModel.add(keras.layers.Dense(numberClasses, activation="softmax"))

lstmModel.compile(loss="categorical_crossentropy", optimizer="rmsprop", metrics=["accuracy"])
lstmModel.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_33 (LSTM)              (None, 3, 128)            393216    
                                                                 
 lstm_34 (LSTM)              (None, 3, 64)             49408     
                                                                 
 lstm_35 (LSTM)              (None, 32)                12416     
                                                                 
 dropout_11 (Dropout)        (None, 32)                0         
                                                                 
 dense_18 (Dense)            (None, 32)                1056      
                                                                 
 dense_19 (Dense)            (None, 16)                528       
                                                                 
 dense_20 (Dense)            (None, 7)                

In [87]:
y_train_tensor = tf.convert_to_tensor(y_train)
y_valid_tensor = tf.convert_to_tensor(y_valid)

x_train_tensor = tf.convert_to_tensor(x_train_lstm)
x_valid_tensor = tf.convert_to_tensor(x_valid_lstm)

# training the base LSTM model
lstmModel.fit(x = x_train_tensor, y = y_train_tensor, batch_size=32, epochs=100, validation_data=(x_valid_tensor, y_valid_tensor))

Epoch 1/100
89/89 [==============================] - 5s 19ms/step - loss: 1.9461 - accuracy: 0.1510 - val_loss: 1.9418 - val_accuracy: 0.2200
Epoch 2/100
89/89 [==============================] - 1s 9ms/step - loss: 1.9410 - accuracy: 0.1542 - val_loss: 1.9467 - val_accuracy: 0.1400
Epoch 3/100
89/89 [==============================] - 1s 9ms/step - loss: 1.9315 - accuracy: 0.1823 - val_loss: 1.9580 - val_accuracy: 0.2000
Epoch 4/100
89/89 [==============================] - 1s 9ms/step - loss: 1.9241 - accuracy: 0.1770 - val_loss: 1.9599 - val_accuracy: 0.1300
Epoch 5/100
89/89 [==============================] - 1s 9ms/step - loss: 1.9149 - accuracy: 0.1978 - val_loss: 1.9323 - val_accuracy: 0.2000
Epoch 6/100
89/89 [==============================] - 1s 9ms/step - loss: 1.9069 - accuracy: 0.2020 - val_loss: 1.9333 - val_accuracy: 0.1900
Epoch 7/100
89/89 [==============================] - 1s 9ms/step - loss: 1.8953 - accuracy: 0.2153 - val_loss: 1.9586 - val_accuracy: 0.2000
Epoch 8/100


### LSTM with Pitch

In [71]:
#data for LSTM for Pitch
x_train_lstm_1 = []
x_train_lstm_2 = []
for dataCollection in np.array(train[["Pitch", "Pitch_Range", "Pitch_Average"]]):
    pitch, pitch_range, pitch_average = dataCollection
    x_train_lstm_1.append([pitch])
    x_train_lstm_2.append([pitch_range, pitch_average])
x_train_lstm_1 = np.array(x_train_lstm_1)
x_train_lstm_2 = np.array(x_train_lstm_2)
print(x_train_lstm_1.shape)

x_valid_lstm_1 = []
x_valid_lstm_2 = []
for dataCollection in np.array(valid[["Pitch", "Pitch_Range", "Pitch_Average"]]):
    pitch, pitch_range, pitch_average = dataCollection
    x_valid_lstm_1.append([pitch])
    x_valid_lstm_2.append([pitch_range, pitch_average])
x_valid_lstm_1 = np.array(x_valid_lstm_1)
x_valid_lstm_2 = np.array(x_valid_lstm_2)
print(x_valid_lstm_1.shape)

(2847, 1, 320)
(100, 1, 320)


In [78]:
timesteps = len(data["Pitch"][0])
numberClasses = 7

LSTMInput_1 = keras.layers.Input(shape=(1, timesteps))
LSTMModel_1 = keras.layers.LSTM(64, return_sequences=True)(LSTMInput_1)
LSTMModel_1 = keras.layers.LSTM(32, return_sequences=True)(LSTMModel_1)
LSTMModel_1 = keras.layers.LSTM(32)(LSTMModel_1)
LSTMModel_1 = keras.layers.Dropout(0.1)(LSTMModel_1)
LSTMModel_1 = keras.layers.Flatten()(LSTMModel_1)

LSTMInput_2 = keras.layers.Input(shape=(2,1))
LSTMModel_2 = keras.layers.LSTM(64, return_sequences=True)(LSTMInput_2)
LSTMModel_2 = keras.layers.LSTM(32, return_sequences=True)(LSTMModel_2)
LSTMModel_2 = keras.layers.LSTM(32)(LSTMModel_2)
LSTMModel_2 = keras.layers.Dropout(0.1)(LSTMModel_2)
LSTMModel_2 = keras.layers.Flatten()(LSTMModel_2)

#Merge two models
Merge = keras.layers.concatenate([LSTMModel_1, LSTMModel_2])

Merge = keras.layers.Dense(32, activation="relu")(Merge)
Merge = keras.layers.Dense(16, activation="relu")(Merge)
Output = keras.layers.Dense(numberClasses, activation="softmax")(Merge)

lstmModel = keras.Model(inputs=[LSTMInput_1, LSTMInput_2], outputs=Output)

lstmModel.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
lstmModel.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 1, 320)]     0           []                               
                                                                                                  
 input_11 (InputLayer)          [(None, 2, 1)]       0           []                               
                                                                                                  
 lstm_24 (LSTM)                 (None, 1, 64)        98560       ['input_10[0][0]']               
                                                                                                  
 lstm_27 (LSTM)                 (None, 2, 64)        16896       ['input_11[0][0]']               
                                                                                            

In [79]:
y_train_tensor = tf.convert_to_tensor(y_train)
y_valid_tensor = tf.convert_to_tensor(y_valid)

x_train_lstm_tensor_1 = tf.convert_to_tensor(x_train_lstm_1)
x_valid_lstm_tensor_1 = tf.convert_to_tensor(x_valid_lstm_1)
x_train_lstm_tensor_2 = tf.convert_to_tensor(x_train_lstm_2)
x_valid_lstm_tensor_2 = tf.convert_to_tensor(x_valid_lstm_2)

lstmModel.fit([x_train_lstm_tensor_1, x_train_lstm_tensor_2], y_train_tensor, batch_size=32, epochs=100, validation_data=([x_valid_lstm_tensor_1, x_valid_lstm_tensor_2], y_valid))

Epoch 1/100
89/89 [==============================] - 8s 27ms/step - loss: 1.9463 - accuracy: 0.1310 - val_loss: 1.9453 - val_accuracy: 0.1200
Epoch 2/100
89/89 [==============================] - 1s 11ms/step - loss: 1.9457 - accuracy: 0.1510 - val_loss: 1.9407 - val_accuracy: 0.2200
Epoch 3/100
89/89 [==============================] - 1s 11ms/step - loss: 1.9456 - accuracy: 0.1475 - val_loss: 1.9443 - val_accuracy: 0.1900
Epoch 4/100
89/89 [==============================] - 1s 11ms/step - loss: 1.9451 - accuracy: 0.1472 - val_loss: 1.9443 - val_accuracy: 0.1900
Epoch 5/100
89/89 [==============================] - 1s 10ms/step - loss: 1.9450 - accuracy: 0.1542 - val_loss: 1.9377 - val_accuracy: 0.2200
Epoch 6/100
89/89 [==============================] - 1s 10ms/step - loss: 1.9456 - accuracy: 0.1482 - val_loss: 1.9416 - val_accuracy: 0.2000
Epoch 7/100
89/89 [==============================] - 1s 10ms/step - loss: 1.9446 - accuracy: 0.1472 - val_loss: 1.9413 - val_accuracy: 0.1900
Epoch 

#### CNN and LSTM

In [28]:
#CNN model
CNNInput = keras.layers.Input(shape=(200, 300, 3))
CNNModel = keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu')(CNNInput)
#CNNModel = keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu')(CNNModel)
CNNModel = keras.layers.BatchNormalization()(CNNModel)
CNNModel = keras.layers.MaxPooling2D(pool_size=(2, 2))(CNNModel)
CNNModel = keras.layers.Dropout(0.10)(CNNModel)
CNNModel = keras.layers.Flatten()(CNNModel)

#LSTM model
data_dimension = 2
timesteps = len(data["Timings"][0])
numberClasses = 7

LSTMInput = keras.layers.Input(shape=(data_dimension, timesteps))
LSTMModel = keras.layers.LSTM(64, return_sequences=True)(LSTMInput)
LSTMModel = keras.layers.LSTM(32, return_sequences=True)(LSTMModel)
LSTMModel = keras.layers.LSTM(32)(LSTMModel)
LSTMModel = keras.layers.Dropout(0.1)(LSTMModel)
LSTMModel = keras.layers.Flatten()(LSTMModel)

#Merge two models
Merge = keras.layers.concatenate([CNNModel, LSTMModel])

Merge = keras.layers.Dense(32, activation="relu")(Merge)
Merge = keras.layers.Dense(16, activation="relu")(Merge)
Output = keras.layers.Dense(numberClasses, activation="softmax")(Merge)

CNN_LSTM_model = keras.Model(inputs=[CNNInput, LSTMInput], outputs=Output)
CNN_LSTM_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [29]:
y_train_tensor = tf.convert_to_tensor(y_train)
y_valid_tensor = tf.convert_to_tensor(y_valid)

x_train_cnn_tensor = tf.convert_to_tensor(x_train_cnn)
x_valid_cnn_tensor = tf.convert_to_tensor(x_valid_cnn)
x_train_lstm_tensor = tf.convert_to_tensor(x_train_lstm)
x_valid_lstm_tensor = tf.convert_to_tensor(x_valid_lstm)

CNN_LSTM_model.fit([x_train_cnn_tensor, x_train_lstm_tensor], y_train_tensor, batch_size=16, epochs=15, validation_data=([x_valid_cnn_tensor, x_valid_lstm_tensor], y_valid_tensor))

Epoch 1/15
178/178 [==============================] - 8s 36ms/step - loss: 21.3836 - accuracy: 0.1370 - val_loss: 1.9458 - val_accuracy: 0.1200
Epoch 2/15
178/178 [==============================] - 6s 34ms/step - loss: 1.9459 - accuracy: 0.1345 - val_loss: 1.9451 - val_accuracy: 0.1200
Epoch 3/15
178/178 [==============================] - 6s 34ms/step - loss: 1.9916 - accuracy: 0.1345 - val_loss: 1.9449 - val_accuracy: 0.2100
Epoch 4/15
178/178 [==============================] - 6s 34ms/step - loss: 1.9458 - accuracy: 0.1489 - val_loss: 1.9443 - val_accuracy: 0.2100
Epoch 5/15
178/178 [==============================] - 6s 34ms/step - loss: 1.9457 - accuracy: 0.1423 - val_loss: 1.9440 - val_accuracy: 0.2100
Epoch 6/15
178/178 [==============================] - 6s 34ms/step - loss: 1.9458 - accuracy: 0.1423 - val_loss: 1.9438 - val_accuracy: 0.2100
Epoch 7/15
178/178 [==============================] - 6s 34ms/step - loss: 1.9457 - accuracy: 0.1489 - val_loss: 1.9442 - val_accuracy: 0.210